# Aula 05 - Aprendizagem não-supervisionada & k-means

Até o momento, neste modelo, já estudamos algumas aplicações de algoritmos do tipo *máquinas de vetores suporte* e vimos alguns métodos de *boosting*. Nestes casos, sempre sabíamos a que classe de dados pertenciam as amostras que gostaríamos de prever, no momento de treinamento dos modelos. Agora, vamos começar a discutir um outro âmbito da área de Aprendizado de Máquina: a **aprendizagem não-supervisionada!**

___

Para iniciar nossa discussão, vamos carregar um [conjunto de dados ilustrativo](https://www.kaggle.com/datasets/samuelcortinhas/time-series-practice-dataset) que traz séries temporais da venda de diversos produtos por algumas lojas.

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

Temos, assim, poucas colunas no nosso conjunto de dados. Vamos fazer um plot, para ter uma ideia dessas quantidades.

In [ ]:
df.groupby("Date")['number_sold'].mean().plot()

In [ ]:
df.groupby(["Date", "store"])['number_sold'].mean().unstack().plot()

In [ ]:
df.groupby(["Date", "product"])['number_sold'].mean().unstack().plot()

A partir dessas visualizações, já conseguimos ter uma ideia de qual o comportamento dos nossos dados. Agora, vamos filtrar apenas um dos anos e tentar observar relações entre algumas variáveis?

In [ ]:
df[df['Date'] < '2011-01-01']

In [ ]:
df[df['Date'] < '2011-01-01'].groupby(["Date", "product"])['number_sold'].mean().unstack().plot()

In [ ]:
sns.scatterplot(data = df[df['Date'] < '2011-01-01'],
               x = 'number_sold',
               y = 'product')

**Pergunta:** a que corresponde cada um dos pontos no plot acima?

In [ ]:
sns.scatterplot(data = df[df['Date'] < '2011-01-01'],
               x = 'number_sold',
               y = 'product',
               hue = 'store',
               palette = 'mako')

Quando adicionamos a informação de a que loja cada conjunto de pontos pertence, podemos entender bem melhor que **lojas específicas possuem faixas específicas de unidades vendidas para cada tipo de produto**. Possivelmente, isso é influenciado por vários fatores, como porte das lojas, capacidade de atrir clientes etc.

Mas e se não soubéssemos, previamente, a que loja pertence cada um dos pontos acima? Será que seria possível tentar fazer essa inferência? 

____
____
_____

## Aprendizagem não-supervisionada

Chegamos ao nosso último tópico do módulo: **aprendizagem não-supervisionada (unsurpervised learning)**.

Este tipo de aprendizagem se diferencia da aprendizagem supervisionada de modo muito simples: **os targets não fazem parte da base de dados!**

> Na aprendizagem não-supervisionada, temos acesso apenas ao conjunto de features, $\{\vec{x}_i\}_{i=1}^N$

A perda que temos com relação à aprendizagem supervisionada é gigante: sem os targets, torna-se impossível a estimação do processo teórico $\mathcal{F}$ que gerou os dados!

Assim, o máximo que podemos fazer na aprendizagem não-supervisionada é a **determinação de estrutura nos dados**:

<img src=https://www.researchgate.net/profile/Zhenyu-Wen-2/publication/336642133/figure/fig3/AS:815304842170368@1571395230317/Examples-of-Supervised-Learning-Linear-Regression-and-Unsupervised-Learning.png width=500>

Em um problema de classificação, somos capazes de encontrar a fronteira de decisão dentre as classes que **são conhecidas no treino**:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/f29c8ebf-dd5f-4fce-99bb-86ec8af21f51.PNG width=700>

Já em problemas não-supervisionados, o máximo que podemos fazer é encontrar a estrutura presente nos dados (e com maior dificuldade!)

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/0c7b530d-e74b-4886-9601-740d054aa166.PNG width=300>

Para muitas aplicações, isso já é suficiente: basta saber que os dados estão estruturados (agrupados/segmentados), sendo o significado de cada grupo/segmento de menor interesse, ou facilmente estimado de outra forma; ou, então, determinar aspectos importantes das features por si só, sem qualquer preocupação com o target.

Neste curso, veremos dois grandes grupos de **técnicas não-supervisionadas**:

- Clusterização - forma de encontrar grupos (clusters) nos dados;
- Redução de dimensionalidade - importante processo de pré-processamento que visa reduzir o número de dimensões (features) de um dataset.

Na aula de hoje, veremos técnicas de clusterização!

______

### Clusterização

Este tipo de problema consiste em __agrupar__ itens semelhantes, isto é, criar __grupos__ (ou __clusters__) dos dados que são parecidos entre si.

> O objetivo central é **dividir os dados em grupos distintos**, tais que **membros de cada grupo sejam similares entre si**

Problemas como estes podem aparecer em diversos contextos:

- Identificação de tipos de clientes parecidos, para o direcionamento de marketing;
- Agrupamento de cidades próximas para melhor logística de entrega de produtos;
- Identificação de padrões climáticos;
- Identificação de genes relacionados à determinada doença;
- Identificação de documentos semelhantes em processos legais;

...e qualquer outro problema em que você deseje **agrupar dados similares** ou ainda **encontrar alguma estrutura nos seus dados!**, mas tudo isso no que diz respeito ùnicamente **às features**!

Veremos agora um dos principais algoritmos de clusterização, o **k-means**



___
___
___

## K-means

Documentação: [clique aqui!](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans)

O k-means é utilizado para a determinação de um número **$k$ de clusters em nossos dados** (mais abaixo explicamos melhor como este algoritmo funciona!)

Para começar a entender um pouco melhor o algoritmo, vamos trabalhar, neste primeiro momento, **com um exemplo ilustrativo**.

In [ ]:
# geração dos dados
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

X, _ = make_blobs(n_samples = 300,
                 n_features = 2,
                 centers = 4,
                 cluster_std = 1,
                 random_state = 42)

plt.scatter(X[:,0], X[:,1])
plt.xlabel("X1")
plt.ylabel("X2")

O primeiro passo pra aplicar o $k$-means é:

- Determinar o número $k$ de clusters!

Por exemplo, só de olhar pros dados plotados a seguir, fica fácil de identificar 4 grupos distintos, não é mesmo? 

In [ ]:
import pandas as pd
pd.DataFrame(X, columns = "x1 x2".split())


Mas, como o computador pode identificar estes grupos? É isso que o algoritmo responde!

Uma vez determinado o número k de clusters, podemos construir nosso modelo!

### Construindo o modelo

Note que temos apenas as **features** dos dados (no caso, $x_1$ e $x_2$). Iso caracteriza um problema de clusterização **não-supervisionado**: quando nossos dados **não têm targets**, apenas features!

In [ ]:
from sklearn.cluster import KMeans

Temos vários argumentos na classe, mas os principais são:

>- n_clusters: quantos clusters queremos (o número k);

>- max_iter: é o número máximos de iterações que o algoritmo fará, se ele não convergir antes disso. É uma boa ideia não colocar um número tão grande, ou o algoritmo pode ficar bem lento. Algo da ordem de 1000, em geral é uma boa escolha.

Por fim, pra fitar o modelo, fazemos:

In [ ]:
# da figura, queremos 4 clusters
kmeans = KMeans(n_clusters = 4, n_init = 'auto')
kmeans

Em algoritmos **não supervisionados**, não existe a divisão em dados de treino e dados de teste, porque **não há o que testar!**. Queremos apenas **econtrar estrutura** nos dados!

Então, basta fitar o modelo com nossos dados todos (no caso, o array X)

In [ ]:
kmeans.fit(X)

Agora que o modelo está treinado, podemos fazer predições:


In [ ]:
kmeans.labels_

In [ ]:
kmeans.predict(X)

In [ ]:
labels_clusters = kmeans.labels_

Isto retorna uma lista com número de elementos igual ao número de pontos do dataset, e com valores entre 0 e k-1, indicando qual é o número do cluster (a contagem começa com zero). 

No nosso caso, como k = 4, teremos os clusters 0, 1, 2 e 3.

Pra visualizarmos os clusters, basta plotar os dados iniciais com o hue adequado!

In [ ]:
X_df = pd.DataFrame(X, columns = "x1 x2".split())
labels_series = pd.Series(labels_clusters, name = "label")

In [ ]:
df_result = pd.concat([X_df, labels_series], axis = 1)
df_result

In [ ]:
import seaborn as sns
sns.scatterplot(data = df_result,
               x = 'x1',
               y = 'x2',
               hue = 'label')

In [ ]:
sns.jointplot(data = df_result, x = 'x1', y = 'x2')

In [ ]:
sns.jointplot(data = df_result, x = 'x1', y = 'x2', hue = 'label')

In [ ]:
# geração dos dados
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

X, _ = make_blobs(n_samples = 300,
                 n_features = 2,
                 centers = 4,
                 cluster_std = 2.5,
                 random_state = 42)

plt.scatter(X[:,0], X[:,1])
plt.xlabel("X1")
plt.ylabel("X2")

In [ ]:
kmeans = KMeans(n_clusters = 4, n_init = 'auto')
kmeans

In [ ]:
kmeans.fit(X)
kmeans.labels_

In [ ]:
kmeans = KMeans(n_clusters = 4, n_init = 'auto') # inicialização
kmeans.fit(X) # fit dos dados

# plotando o resultado
X_df = pd.DataFrame(X, columns = "x1 x2".split())
labels_clusters = kmeans.labels_
labels_series = pd.Series(labels_clusters, name = "label")
df_result = pd.concat([X_df, labels_series], axis = 1)
df_result

In [ ]:
sns.jointplot(data = df_result, x = 'x1', y = 'x2', hue = 'label')

Na prática, a análise segue de maneira qualitativa, inspecionando os clusters individuais:

In [ ]:
df_result.query("label == 0").describe()

In [ ]:
sns.jointplot(data = df_result.query("label == 0"), x = 'x1', y = 'x2')

_____

### Determinando o $k$

Mas e se não for tão fácil de plotar os dados para determinar o $k$?

Pode ser que não consigamos visualizar nossos dados em 2D, se, por exemplo, tivermos mais de 2 features em nossos dados...

> Quase sempre, uma boa metodologia para a determinaçãodo número de clusters é **conhecimento do negócio**! Muitas vezes, o próprio problema nos indica a quantidade de clusters que esperamos encontrar!

No entanto, há situações em que o número de clusters não é conhecido a priori.

Neste caso, podemos usar o __método do cotovelo__, que consiste em rodar o k-means várias vezes, para diferentes valores de k, e depois plotar um gráfico com a **inércia** de cada uma das rodadas. 

### Inércia (WCSS) e método do cotovelo

A inércia também é chamada de **WCSS** (Within-Cluster-Sum-of-Squares), isto é, "soma de quadrados intra-cluster", que é calculada como a soma das distâncias (ao quadrado) entre os pontos e os centróides dos clusters.

Quanto menor o WCSS, mais eficiente foi a clusterização, **mas até certo ponto!**

Conforme o número de clusters ($k$) aumenta, o WCSS diminui, sendo mínimo quando cada ponto é seu próprio cluster isolado (o que não é nada útil, pois se cada ponto for um cluster, não há clusterização alguma!).

Assim, o que queremos não é encontrar um $k$ que minimize o WCSS, mas sim um k a partir do qual o WCSS **para de decrescer tão rapidamente!**

Quando encontramos este $k$, encontramos o número ideal de clusters!

Ao plotarmos o WCSS (inércia) em função de $k$, o que buscaremos será então o valor de $k$ após o qual **o gráfico deixa de ser tão inclinado**. Esses pontos são visualizados como "quinas", ou **cotovelos** no gráfico -- e daí vem o nome do método!

Para aplicar o método, fazemos:
 

In [ ]:
kmeans.inertia_

Vamos, agora, escrever uma função para calcular a inércia para vários valores de *k*.

In [ ]:
# geração dos dados
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

X, _ = make_blobs(n_samples = 300,
                 n_features = 2,
                 centers = 4,
                 cluster_std = 1,
                 random_state = 42)

plt.scatter(X[:,0], X[:,1])
plt.xlabel("X1")
plt.ylabel("X2")

In [ ]:
def calc_inercias(X, lista_k, plot=True):
    lista_inercias = []
    
    X_df = pd.DataFrame(X, columns = [f"X{i+1}" for i in range(X.shape[1])])
    
    for k in lista_k:
        kmeans = KMeans(n_clusters = k)
        kmeans.fit(X)
        
        labels_clusters = kmeans.labels_
        
        # Cálculo das inércias
        inercia = kmeans.inertia_
        lista_inercias.append(inercia)
        
        labels_series = pd.Series(labels_clusters, name = 'label')
        df_result = pd.concat([X_df, labels_series], axis = 1)
        
        # Vamos, também, plotar como ficam os agrupamentos
        if plot and X.shape[1] == 2:
            print(f"\nInércia para clusterização com k = {k}: {inercia}")
            
            sns.jointplot(data = df_result,
                        x = 'X1',
                        y = 'X2',
                        hue = 'label')
            plt.show()
    return lista_inercias

In [ ]:
inercias = calc_inercias(X, [1, 2, 3, 4, 5, 6, 7, 8], plot = True)

In [ ]:
inercias

In [ ]:
# função para plotar as inércias
def plot_cotovelo(lista_k, lista_inercias):
    plt.figure(figsize=(8,5))
    
    plt.title('Método do cotovelo')
    plt.plot(lista_k, lista_inercias, marker = "o")
    
    plt.xlabel("k")
    plt.ylabel("Inércia (WCSS)")
    
    plt.show()

In [ ]:
np.linspace(1,8,8)

In [ ]:
import numpy as np
plot_cotovelo(list(np.linspace(1,8,8)), inercias)

In [ ]:
# geração dos dados
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

X, _ = make_blobs(n_samples = 300,
                 n_features = 2,
                 centers = 4,
                 cluster_std = 2.5,
                 random_state = 42)

plt.scatter(X[:,0], X[:,1])
plt.xlabel("X1")
plt.ylabel("X2")

In [ ]:
inercias = calc_inercias(X, [1, 2, 3, 4, 5, 6, 7, 8], plot = False)

In [ ]:
plot_cotovelo(list(np.linspace(1,8,8)), inercias)

O valor de $k$ mais adequado é aquele em que o gráfico tem uma "quina" bem abrupta: no exemplo acima, $k = 4$, como já sabíamos!

_______

### Método da silhueta

Um método alternativo ao método do cotovelo para o cálculo do número adequado de clusters é o método da silhueta.

Neste método, é calculado para cada ponto um score conhecido como **coeficiente de silhueta**, que é dado por:

$$ s = \frac{b - a}{max(a, b)} $$

onde:

- $a$ é a **distância média entre um dado ponto e os pontos de seu próprio cluster**. Portanto, essa é uma medida de **similaridade entre um ponto e seu cluster**;
- $b$ é a **distância média entre um dado ponto e os pontos do cluster mais próximo (sem ser o próprio).** Portanto, essa é uma medida de **dissimilaridade entre um ponto e os demais clusters**;

Graficamente:

<img src=https://miro.medium.com/max/712/1*cUcY9jSBHFMqCmX-fp8BvQ.jpeg width=400>

Note que $-1 < s < 1$, sendo mais próximo de $1$ quando um ponto está no cluster correto ($a \ll b$); e mais próximo de $-1$ quando um ponto está no custer errado ($b \gg a$).

Na prática, é costumeiro olhar para **a média do coeficiente $s$ para todos os pontos, denotado $\bar{s}$**, e apresentar uma única métrica. A ideia é que se, em média, tivermos pontos em clusters corretos, teremos $\bar{s} \rightarrow 1$; enquanto, se em média tivermos muitos pontos em clusters incorretos, teremos $\bar{s} \rightarrow -1$.

Este score é calculado com a função [silhouette_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) do sklearn.

Uma vez que é possível calcularmos o score para um dado $k$, a decisão sobre o melhor $k$ segue similar ao método do cotovelo: basta calcular o score de silhueta para vários valores de $k$, e selecionar aquele que dá **a silueta mais próxima de $1$**!

Vamos fazer isso, abaixo?

In [ ]:
from sklearn.metrics import silhouette_score

def calc_silhueta(X, lista_k, plot = True):
    lista_silhuetas = []
    
    X_df = pd.DataFrame(X, columns = [f"X{i+1}" for i in range(X.shape[1])])
    
    for k in lista_k:
        kmeans = KMeans(n_clusters = k)
        kmeans.fit(X)
        
        labels_clusters = kmeans.labels_
        
        # Cálculo das silhuetas
        silhueta = silhouette_score(X, labels_clusters)
        lista_silhuetas.append(silhueta)
        
        labels_series = pd.Series(labels_clusters, name = 'label')
        df_result = pd.concat([X_df, labels_series], axis = 1)
        
        # Vamos, também, plotar como ficam os agrupamentos
        if plot and X.shape[1] == 2:
            print(f"\nSilueta média para clusterização com k = {k}: {silhueta}")
            
            sns.jointplot(data = df_result,
                        x = 'X1',
                        y = 'X2',
                        hue = 'label')
            plt.show()
    return lista_silhuetas

In [ ]:
lista_k = range(2,9)

lista_silhuetas = calc_silhueta(X, lista_k, plot = False)

In [ ]:
def plot_silhueta(X, lista_k):
    
    lista_silhuetas = calc_silhueta(X, lista_k, plot=False)

    plt.figure(figsize=(8, 5))

    plt.title("Método da silhueta")

    plt.plot(lista_k, lista_silhuetas, marker="o")

    plt.xlabel("k (# de clusters)")
    plt.ylabel("Mean silhouette score")

    plt.show()

In [ ]:
plot_silhueta(X, lista_k)

In [ ]:
sns.scatterplot(data = X_df,
               x = 'x1',
               y = 'x2')

Aqui novamente, fica claro que o ideal é $k=3$!

Para entender o porquê do método receber o nome "silhueta", podemos utilizar o seguinte código do sklearn, [que tirei daqui!](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html)

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import numpy as np

import matplotlib.cm as cm

range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    
    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    title = "Silhouette analysis for KMeans clustering on sample data "
    title += "with n_clusters = {}\nSilhouette score = {:.2f}".format(n_clusters, silhouette_avg)
    plt.suptitle(title, fontsize=14, fontweight='bold')

    plt.show()
    
    print("\n\n")

Na prática, é recomendável usar ambos os métodos, do cotovelo e da silhueta, pra apoiar a tomada de decisão quanto ao valor adequado de $k$.

No entanto, lembre-se: sempre que possível, guie esta decisão segundo o contexto do problema de negócio!

___

### Vamos fazer um exemplo com mais features

In [ ]:
from sklearn.datasets import make_blobs
X_new, _ = make_blobs(n_samples = 500,
                     n_features = 4,
                     centers = 6,
                     cluster_std = 1,
                      random_state = 0
                     )
df = pd.DataFrame(X_new, columns = [f"X{i+1}" for i in range(X_new.shape[1])])

In [ ]:
df

In [ ]:
sns.pairplot(data = df)

**Conclusão:** muito difícil de escolher o $k$ com base em análise exploratória!

Aí, não tem jeito, temos que utilizar os métodos quantitativos do cotovelo e/ou silhueta!

**Aplicando o método do cotovelo...**

In [ ]:
inercias = calc_inercias(X_new, range(2,11))
plot_cotovelo(range(2,11), inercias)

**Aplicando o método da silhueta...**

In [ ]:
plot_silhueta(X_new, range(2,11))

**Vamos tentar separadamente $k=6$**

In [ ]:
kmeans = KMeans(n_clusters = 6)
kmeans.fit(X_new)

labels = kmeans.labels_
X_new_df = pd.DataFrame(X_new, columns = [f"X{i+1}" for i in range(X_new.shape[1])])
dados_clusterizados = pd.concat([X_new_df, 
                                 pd.Series(labels, name = 'clusters')],
                               axis = 1)

sns.pairplot(data = dados_clusterizados,
            hue = 'clusters')

As projeções em duas dimensões mostram que $k=6$ de fato é a melhor escolha! (O que faz sentido, pois nossos dados artificiais foram preparados para conter 6 clusters!)

____

### E como o k-means funciona?

Uma vez escolhido o número de clusters, o k-means segue as seguintes etapas:

- 1) k pontos são escolhidos aleatoriamente como sendo os centroides dos clusters (centroide é o centro do cluster);

- 2) Para cada ponto, vamos calcular qual é a distância entre ele e os k centroides. Aquele centroide que estiver mais perto, será o cluster ao qual este ponto pertencerá. Fazemos isso para todos os pontos!

- 3) Ao fim do passo 2, teremos k clusters, cada um com seu centroide, e todos os pontos pertencerão a determinado cluster!

- 4) Uma vez que temos os clusters, calculamos qual é de fato o centro de cada um deles. Isso é feito tomando a média da posição de todos os pontos;

- 5) Após determinar os novos k centroides, repetimos o processo!

- 6) E o processo se repete até que os centroides não mudem mais. Quando esta convergência for alcançada (ou após o número determinado de iterações), o algoritmo termina!

<img src="https://stanford.edu/~cpiech/cs221/img/kmeansViz.png" width=700>

<img src="https://miro.medium.com/max/1280/1*rwYaxuY-jeiVXH0fyqC_oA.gif" width=500>

<img src="https://miro.medium.com/max/670/1*JUm9BrH21dEiGpHg76AImw.gif" width=500>

_____

### **Quando uso algoritmos de clusterização, e em que casos eles não são uma boa ideia?**


De certa fora, algoritmos de clusterização podem ser vistos como classificadores, uma vez que os clusters podem caracterizar um grupo, ou uma classe.

No entanto, há uma diferença bem importante entre problemas de classificação e clusterização:

- **Problemas de classificação** são **supervisionados**, isto é, as amostras de treino que utilizamos têm tanto as features como os **targets**. Em outras palavras, neste tipo de problema, sabemos de antemão quais são as classes de interesse - Isto é, temos $\{\vec{x}_i, y_i \}_{i=1}^N$; <br><br>

- **Problemas de clusterização**, por outro lado, são **não-supervisionados**. Ou seja, a amostra **não contêm** targets, temos apenas as features! O nosso objetivo é justamente descobrir **alguma estrutura de agrupamento** nos dados, mas sem qualquer informação prévia quanto aos grupos a serem formados.

Foi exatamente o caso do nosso exemplo: nós tínhamos apenas as **features** dos dados, e **nenhuma** informação quanto aos grupos que seriam formados.

Foi só depois que fizemos a análise exploratória dos dados (plot), que pudemos identificar alguma estrutura (4 clusters), para então aplicar o k-means!

No segundo caso, só pudemos determinar o número de clusters de forma segura utilizando o **método do cotovelo**.

Assim sendo, via de regra, a utilização ou não de algoritmos de clusterização, além do tipo de problema, depende dos **dados disponíveis**!

Além do k-means, há outros algoritmos de clusterização que são muito utilizados, e que se baseiam em princípios bem diferentes do k-means.

Na aula que vem, olharemos para um algoritmo bem importante: [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN).

____
____
____

## Exemplo real

Vamos agora ver o KMeans aplicado a um problema e dataset real!

### Primeiro dataset -> classificação de tipos de vinho a partir da composição química.
[Link para o dataset](https://archive.ics.uci.edu/ml/datasets/wine)

In [ ]:
wine_data = pd.read_csv('wine_data.csv')
wine_data.head()

In [ ]:
wine_data.Class.unique()

A coluna 'Class' nos fornece o tipo do vinho, isso é o que queremos fazer com nosso modelo, por isso, vamos remover essa coluna e salvá-la para analisar os resultados obtidos pelo algoritmo.

In [ ]:
resposta = wine_data['Class']
wine_data.drop(columns = ['Class'], inplace = True)

In [ ]:
resposta

In [ ]:
wine_data.describe()

É possível notar que cada coluna tem valores numéricos com proporções distintas, como o algoritmo funciona medindo distâncias, é melhor que os dados estejam num mesmo alcance (de 0 a 1 por exemplo). Como sabemos, o sklearn tem um algoritmo de préprocessamento que faz isso.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
transformed_wine = scaler.fit_transform(wine_data)
transformed_wine

In [ ]:
plt.figure(figsize = (13,8))
sns.heatmap(wine_data.corr(), annot = True)

Agora com os valores entre 0 e 1 vamos instanciar o modelo.

In [ ]:
sns.pairplot(data = wine_data)

In [ ]:
inercias = calc_inercias(transformed_wine, range(2,11))
plot_cotovelo(range(2,11), inercias)

In [ ]:
plot_silhueta(transformed_wine, range(2,11))

In [ ]:
df_transform = pd.DataFrame(transformed_wine, columns = wine_data.columns)

In [ ]:
estimador = KMeans(n_clusters = 3, random_state = 42)
modelo = estimador.fit(df_transform)

In [ ]:
modelo.labels_

Ou, podemos usar o .predict() como já estamos acostumados

In [ ]:
modelo.predict(df_transform)

In [ ]:
df_transform['cluster'] = modelo.labels_

In [ ]:
df_transform.head()

In [ ]:
sns.pairplot(data = df_transform, hue = 'cluster')

In [ ]:
df_transform.head()

In [ ]:
sns.scatterplot(data = df_transform,
               x = 'Alcohol',
               y = 'Color intensity',
               hue = 'cluster')

In [ ]:
modelo.cluster_centers_

In [ ]:
resposta[0:10]

In [ ]:
df_transform['cluster'][0:10]

In [ ]:
idx = df_transform[df_transform['cluster'] == 1]['cluster'].index
idx

In [ ]:
resposta[idx].value_counts()

In [ ]:
resposta[165:177]

In [ ]:
df_transform['classe'] = df_transform['cluster']
df_transform.head()

In [ ]:
# vamos contar quantas classificações erradas teríamos?
df_transform.loc[df_transform['cluster'] == 0, 'classe'] = 2
df_transform.loc[df_transform['cluster'] == 1, 'classe'] = 3
df_transform.loc[df_transform['cluster'] == 2, 'classe'] = 1

In [ ]:
df_transform.head()

In [ ]:
sum(abs(resposta - df_transform['classe']))

In [ ]:
df_transform.shape[0]

In [ ]:
9/df_transform.shape[0]

Com isso conseguimos ver quantas vezes nosso modelo errou, dado que eram 178 dados inicialmente. Assim, podemos refletir se precisávamos de mais informações para realizar um cluster melhor ou se o agrupamento correto não apresentava a simetria radial que o KMeans traz.

Será que o método do K-means funcionaria bem para o dataset do início da aula? Fica como exercício!